This is my first kaggle competition using FastAI

**Note** This notebook is incomplete with loads of comments & test code. I am just trying to play around and learn. If there are better way to do things then please comment below. 

v3: using Keras and EfficientNet as a model - public LB pending

v2: used resnet50 with 12 epochs -- public LB 0.103 score - no difference, need to change the image model 

v1: used resnet50 with default params and 3 epochs -- public LB 0.103 score

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import libraries 

In [ ]:
import os, warnings
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

# import fastai; 
# from fastai.vision.all import *
# from fastai.vision.learner import cnn_learner, create_head, create_body, num_features_model, default_split, has_pool_type, apply_init, load_learner

# %config IPCompleter.greedy=True

# fastai.__version__

print("Libraries loaded...")


# Load the files

Load the json

In [ ]:
BASE_DIR = "../input/cassava-leaf-disease-classification/"

# read the json file with disease infection 
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    
print(json.dumps(map_classes, indent=4))

# Load the train dataset via csv

In [ ]:
df_train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

df_train["class_name"] = df_train["label"].astype(str).map(map_classes)
df_train["label"] = df_train["label"].astype(str)

df_train

In [ ]:
df_train.groupby(df_train.label).count()

In [ ]:
train_img = (os.path.join(BASE_DIR, "train_images"))
test_img = (os.path.join(BASE_DIR, "test_images"))
#img_path = '../input/cassava-leaf-disease-classification/train_images/'
train_img, test_img
# img_files = get_image_files(img)
# img_files

# Using Keras and EfficientNet 

In [ ]:
imageDataGenerator = ImageDataGenerator(rescale=1/255.,
                                        rotation_range=45,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        zoom_range=0.2,
                                        shear_range=10,
                                        validation_split=0.2,
                                       )

train_set = imageDataGenerator.flow_from_dataframe(dataframe=df_train,
                                                    directory=train_img,
                                                    x_col="image_id",
                                                    y_col="label",
                                                    target_size=(100, 100),
                                                    batch_size=32,
                                                    class_mode="sparse",
                                                    subset='training',
                                                    shuffle=True,
                                                    seed=42
                                                )

                                       
val_set = imageDataGenerator.flow_from_dataframe(dataframe=df_train,
                                                    directory=train_img,
                                                    x_col="image_id",
                                                    y_col="label",
                                                    target_size=(100, 100),
                                                    batch_size=32,
                                                    class_mode="sparse",
                                                    subset='validation',
                                                    shuffle=True,
                                                    seed=42
                                                )



# Prepare the model

In [ ]:
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.optimizers import Adam

# MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1"
# MODULE_HANDLE = tf.keras.applications.EfficientNetB4(include_top=True, weights='imagenet', input_tensor=None,input_shape=None)

# IMAGE_SIZE = (380, 380)

# loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

# model = tf.keras.Sequential([
#     InputLayer(input_shape=IMAGE_SIZE + (3,)),
#     hub.KerasLayer(MODULE_HANDLE, trainable=False),
#     Dense(512, activation='relu'),
#     Dropout(0.33),
#     Dense(256, activation='relu'),
#     Dropout(0.33),
#     Dense(128, activation='relu'),
#     Dropout(0.33),
#     Dense(5, activation='softmax')
# ])

# model.compile(optimizer=Adam(), loss=loss_func, metrics=['acc'])
# # model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['acc'])

# model.summary()

In [ ]:
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.optimizers import Adam

MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1"

IMAGE_SIZE = (380, 380)

loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

model = tf.keras.Sequential([
    InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=False),
    Dense(512, activation='relu'),
    Dropout(0.33),
    Dense(256, activation='relu'),
    Dropout(0.33),
    Dense(128, activation='relu'),
    Dropout(0.33),
    Dense(5, activation='softmax')
])

model.compile(optimizer=Adam(), loss=loss_func, metrics=['acc'])
# model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['acc'])

model.summary()

# Now lets train the model 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = 'model.h5'
mc = tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, verbose=1, 
                                        save_weights_only=True, monitor='val_loss', 
                                        mode='auto', save_best_only=True)

history = model.fit(train_set, epochs=30, verbose=1, callbacks=[mc], validation_data=val_set)

# Check training and validation loss

In [ ]:
history.history

In [ ]:
history_df =  pd.DataFrame(history.history) #history.history

fig, (ax1, ax2) = plt.subplots(2, 1, sharex='col', figsize=(20, 14))

ax1.plot(history_df['loss'], label='Train loss')
ax1.plot(history_df['val_loss'], label='Validation loss')
ax1.legend(loc='best')
ax1.set_title('Loss')

ax2.plot(history_df['acc'], label='Train accuracy')
ax2.plot(history_df['val_acc'], label='Validation accuracy')
ax2.legend(loc='best')
ax2.set_title('Accuracy')

plt.xlabel('Epochs')
#sns.despine()
plt.show()

# Lets predict and save the results to the file

In [ ]:
# model.save('./best_model.h5')
# #model.save('/kaggle/input/efficientnetleaf/best_model.h5') ../input/efficientnetleaf/model_new.h5
# print('Model saved!')
model.save_weights('./model_weight.h5')

# Inference over the dataset

In [ ]:
myModel = tf.keras.models.load_model('../input/efficientnetleaf/model_new.h5',custom_objects={'KerasLayer':hub.KerasLayer})
print("Model loaded")

In [ ]:
myModel.summary()


In [ ]:
import cv2 
test_img_path = "../input/cassava-leaf-disease-classification/test_images/2216849948.jpg"

img = cv2.imread(test_img_path)
resized_img = cv2.resize(img, (255, 255)).reshape(-1, 255, 255, 3)/255

plt.figure(figsize=(8,4))
plt.title("TEST IMAGE")
plt.imshow(resized_img[0])

In [ ]:
imageDataGenerator = ImageDataGenerator(rescale=1/255.)
test_gen = imageDataGenerator.flow_from_directory('../input/cassava-leaf-disease-classification/test_images/',classes=['0', '1', '2', '3','4'], target_size=(380, 380),
                                                  class_mode=None, batch_size=32, shuffle=False)

# File submission to Kaggle

In [ ]:
IMG_SIZE = 320


mydict = {"image_id":[], 'label':[]}

for img in glob.glob("../input/cassava-leaf-disease-classification/test_images/*.jpg"):
        test_filename = os.path.basename(img)
        print(img)
        img = tf.keras.preprocessing.image.load_img(img)
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
        img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
        probabilities = myModel.predict(img/255)
        #mydict.append(np.argmax(prediction))
        predictions = np.argmax(probabilities, axis=-1)
        mydict["image_id"].append(test_filename)
        mydict["label"].append(predictions[0])
    
df = pd.DataFrame(mydict)
df.to_csv('submission.csv', index=False)
!head submission.csv
print("File submitted")

# FastAI method

In [ ]:
#dls = ImageDataLoaders.from_df(df=df_train, path=train_img, seed=16, item_tfms=Resize(460), batch_tfms=aug_transforms(size=224))
# learn = cnn_learner(dls, resnet50, metrics=accuracy) 
# learn.fit_one_cycle(3, 3e-3)
# learn.unfreeze()
# learn.fit_one_cycle(12, lr_max=slice(1e-6,1e-4))


In [ ]:
# learn.model_dir = "/kaggle/working"
# learn.lr_find()

In [ ]:
#learn.recorder.plot_loss()
#Learn.model_dir = "/kaggle/working"


# Fine tuning

Lets fine tune the model so that intead of accepting ramdom weight we apply specific weights for our model

# Export the model 

we should export the model to use the competition as our "Internet" needs to be off

In [ ]:
# interp = ClassificationInterpretation.from_learner(learn)
# interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

# learn.export('/kaggle/working/model.pkl')
# print("Model exported")

# Making predictions 

In [ ]:

# learn_inf = load_learner("/kaggle/input/cassavaleafmodel/model.pkl")
# print("Model loaded")

# mydict = {"image_id":[], 'label':[]}
# for img in glob.glob("../input/cassava-leaf-disease-classification/test_images/*.jpg"):
#         test_filename = os.path.basename(img)
#         probabilities = learn_inf.predict(img)
#         predictions = np.argmax(probabilities, axis=-1)
#         #print(test_filename, predictions)
#         mydict["image_id"].append(test_filename)
#         mydict["label"].append(predictions)
    


# df = pd.DataFrame(mydict)
# df.to_csv('submission.csv', index=False)
# !head submission.csv

# Attempt 2: EfficientNet and custom weights using FastAI

lets see if we can use EfficientNet & FastAI to get a better result

1. Install Ross Wightman repository to download efficeientNet model (instlal timm)
2. Use custom weights to use with FASAI


In [ ]:
# def create_timm_body(arch:str, pretrained=True, cut=None):
#   model = create_model(arch, pretrained=pretrained)
#   if cut is None:
#     ll = list(enumerate(model.children()))
#     cut = next(i for i,o in reversed(ll) if has_pool_type(o))
#   if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
#   elif callable(cut): return cut(model)
#   else: raise NameError("cut must be either integer or function")

In [ ]:
# net = create_model('efficientnet_b3a', pretrained=True)
# net

In [ ]:

# body = create_timm_body('efficientnet_b3a', pretrained=True)
# len(body)

In [ ]:
# body

In [ ]:
# nf = num_features_model(nn.Sequential(*body.children())) * (2); nf

In [ ]:
# train_img = (os.path.join(BASE_DIR, "train_images"))
# print(train_img)
# dls = ImageDataLoaders.from_df(df=df_train, path=train_img, seed=16, item_tfms=Resize(460), batch_tfms=aug_transforms(size=224))

In [ ]:
# dls.show_batch(max_n=9,figsize=(8,9))

In [ ]:
# head = create_head(nf, dls.c)
# head

In [ ]:
# model = nn.Sequential(body, head)
# apply_init(model[1], nn.init.kaiming_normal_)
# len(model)

In [ ]:
# learn = Learner(dls, model,loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)
# learn.summary()

In [ ]:
# learn.model_dir = "/kaggle/working"
# learn.lr_find()

In [ ]:
# learn.fit_one_cycle(20, slice(3e-2)) # 0.78 accuracy

In [ ]:
# learn.save('stage_1')

In [ ]:
# learn.unfreeze()
# learn.lr_find()

In [ ]:
#learn.fit_one_cycle(5, 1e-4)

In [ ]:
#learn.save('model_2')

In [ ]:
#learn.export('/kaggle/working/model.pkl')
#print("Model exported")
#learn_inf = load_learner("/kaggle/input/efficientnetleaf/model.pkl")
#print("Model loaded")

# mydict = {"image_id":[], 'label':[]}
# for img in glob.glob("../input/cassava-leaf-disease-classification/test_images/*.jpg"):
#         test_filename = os.path.basename(img)
#         probabilities = learn_inf.predict(img)
#         predictions = np.argmax(probabilities, axis=-1)
#         #print(test_filename, predictions)
#         mydict["image_id"].append(test_filename)
#         mydict["label"].append(predictions)
    


# df = pd.DataFrame(mydict)
# df.to_csv('submission.csv', index=False)
# !head submission.csv